This example show how to use py2gcode to generate gcode for create lines from an svg file, with special focus in the generation of gcode for fusing lines used in inflatables fabric actuators. 

So the workflow in this notebooks works as follows:

1. Load the svg file
2. Explore the files, and find the curves.
3. Translate this vector description to coordinates.
4. Use py2gcode to create our custom made gcode. 
